In [1]:
from tremlib import TremResearcher
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
researcher = TremResearcher(marks_num=5, marks_size_mm=10)

In [2]:
if __name__ == '__main__':
    from multiprocessing import Pool
    import time
    import os
    from functools import partial
    
    calib_images_path = fr'D:\Projects\Tremor\Calibration\Calibration photo\Accurate attempt 5'
    calib_images = np.asarray(TremResearcher.photo_files(calib_images_path))
    files_to_check = len(calib_images)
    
    
    def f(calib_images_path, step_num):
        from tremlib import TremResearcher
        import numpy as np
        import cv2
        
        FRAMES_PER_STEP = 3
        STEP_MM = 5
        
        allowable_error = np.arange(5, 0, -5/60)[step_num]
        
        _researcher = TremResearcher(marks_num=5, marks_size_mm=10)
        calib_images = np.asarray(_researcher.photo_files(calib_images_path))
        
        check_img_paths_X = _researcher.\
                             photo_files(r'D:\Projects\Tremor\Calibration\Check calibration photo\Focus distance 50 mm 2\X')
        check_img_paths_Y = _researcher.\
                             photo_files(r'D:\Projects\Tremor\Calibration\Check calibration photo\Focus distance 50 mm 2\Y')
        
        errors = np.loadtxt('Errors for Accurate attempt 5 XY (2).txt')
        x_errors, y_errors = errors
        bool_indexes = np.logical_and(x_errors < allowable_error, y_errors < allowable_error)
        
        if len(calib_images[bool_indexes]) == 0:        
            return None
        
        with open(r'Logs\Logs_of_%2d_thread.txt' % step_num, 'w') as f:
            # import sys
            # stdout_back = sys.stdout
            # sys.stdout = f
            print(f"Calibrate sample with allowable_error={allowable_error}. Started.\n")
            _researcher.calibrate(images_paths=calib_images[bool_indexes])
            print(f"Calibrate sample with allowable_error={allowable_error}. Finished.\n")
        
            def mean_and_track(check_img_paths):
                print(f"Recognition with allowable_error={allowable_error}. Started.\n")
                
                if len(check_img_paths) != 0:
                    print("Folder: %s" % check_img_paths[0].split('\\')[0])
                
                postions = []
                for image_path in check_img_paths:
                    frame = cv2.imread(image_path)
                    postions += list(_researcher.get_xyz(frame)[0].values())
                print(f"Recognition with allowable_error={allowable_error}. Finished.\n")
    
                postions = np.asarray(postions)
                postions -= postions[0]
    
                length = [np.linalg.norm(position) for position in postions]
                r = length - np.asarray([i*STEP_MM for i in range(len(length)//FRAMES_PER_STEP) for j in range(FRAMES_PER_STEP)])
    
                return np.mean(r), r
            mean_x, sigma_x = mean_and_track(check_img_paths_X)
            mean_y, sigma_y = mean_and_track(check_img_paths_Y)
    
            _res = {'mean_x' : mean_x,
                    'sigma_x': sigma_x,
                    'mean_y' : mean_y,
                    'sigma_y': sigma_y}
            print(f"allowable error: {allowable_error} RES: {_res}")
            # sys.stdout = stdout_back
        
        return _res   

    with Pool(1) as pool:
        
        new_f = partial(f, calib_images_path)
        res_f = pool.map(new_f, range(60))
        
        rang = np.arange(5, 0, -5/60)
        res = list(zip(rang, res_f))


NameError: name 'researcher' is not defined

In [4]:
X = []
Y = []
acc = []
for accuracy, dic in res:
    if dic is not None:
        acc += [accuracy]
        X += [dic['mean_x']]
        Y += [dic['mean_y']]
    #print(f"Accuracy:{accuracy} X:{tup[0][0]} y:{tup[1][0]}")
acc = acc[::-1]
X = X[::-1]
Y = Y[::-1]
r = 60
plt.scatter(acc[:r], X[:r])
plt.scatter(acc[:r], Y[:r])
plt.plot(acc[:r], [0]*len(acc[:r]))
plt.show()

NameError: name 'res' is not defined

In [16]:
for x in list(zip(acc, X, Y)):
    print("%.4f X:%.4f Y:%.4f" % x)

5.0000 X:0.5815 Y:1.3078
4.9167 X:0.5815 Y:1.3078
4.8333 X:0.5815 Y:1.3078
4.7500 X:0.5815 Y:1.3078
4.6667 X:0.5815 Y:1.3078
4.5833 X:0.5851 Y:1.2911
4.5000 X:0.5851 Y:1.2911
4.4167 X:0.5892 Y:1.2753
4.3333 X:0.5892 Y:1.2753
4.2500 X:0.5892 Y:1.2753
4.1667 X:0.5892 Y:1.2753
4.0833 X:0.5892 Y:1.2753
4.0000 X:0.5892 Y:1.2753
3.9167 X:0.5768 Y:1.2840
3.8333 X:0.5768 Y:1.2840
3.7500 X:0.5768 Y:1.2840
3.6667 X:0.5659 Y:1.2832
3.5833 X:0.5565 Y:1.2913
3.5000 X:0.5683 Y:1.2692
3.4167 X:0.5683 Y:1.2692
3.3333 X:0.5683 Y:1.2692
3.2500 X:0.5683 Y:1.2692
3.1667 X:0.5769 Y:1.2495
3.0833 X:0.5769 Y:1.2495
3.0000 X:0.5596 Y:1.2562
2.9167 X:0.5663 Y:1.2380
2.8333 X:0.5663 Y:1.2380
2.7500 X:0.5677 Y:1.2258
2.6667 X:0.5723 Y:1.2086
2.5833 X:0.5723 Y:1.2086
2.5000 X:0.5753 Y:1.1935
2.4167 X:0.5684 Y:1.2012
2.3333 X:0.5533 Y:1.1921
2.2500 X:0.5496 Y:1.1885
2.1667 X:0.5460 Y:1.1851
2.0833 X:0.5312 Y:1.1600
2.0000 X:0.5211 Y:1.1637
1.9167 X:0.5148 Y:1.1499
1.8333 X:0.4658 Y:1.1137
1.7500 X:0.4860 Y:1.1080
